Regression with "deep" tensor product
JLF May 2018

In [1]:
import os
import numpy as np
import pandas
import math
from random import randint

/Users/jean-loup-faulon/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib-2.2.2+828.ga2f65a60e-py3.5-macosx-10.7-x86_64.egg/matplotlib/__init__.py:961: UserWarning: could not find rc file; returning defaults
  warnings.warn(message)


In [2]:
# FUNCTIONS TO READ SEQUENCE AND ACTIVITY DATA
import Bio
from rdkit.Chem import AllChem
from rdkit.Chem.inchi import MolFromInchi

def getInchi(filename): 
    # Return a dictonary InChIs{MNX:Inchi,..}
    # Change this function if
    # ID/name is not in row[0] and InChI not in row[5]
    InChIs = {}
    with open(filename) as h:
        for line in h:
            if line.startswith('#'):
                continue
            row = line.rstrip().split('\t')
            mnx = row[0]
            InChIs[mnx] = row[5]
    return InChIs

def fasta_reader(filename):
  from Bio.SeqIO.FastaIO import FastaIterator
  with open(filename) as handle:
    for record in FastaIterator(handle):
      yield record

def get_seq(ID):
    # This function exract sequences from ebi
    command  = "curl -k -X GET --header 'Accept:text/x-fasta' 'https://www.ebi.ac.uk/proteins/api/proteins/" 
    command += str(ID) + "'"
    command += " > JLF-2018-tmp.fasta"
    os.system(command)
    s =""
    for entry in fasta_reader("JLF-2018-tmp.fasta"):
        s = str(entry.seq)
    return s

def read_data_file(data_file_name,chem_file_name):
    # Change based on the file format must contain
    # a chemical id, a sequence id and an activity
    # return molecules, sequences and activities
    N = 0
    EC = {}
    ID = {}
    CP = {}
    RX = {}
    KM = {}
    MOL = {}
    SEQ = {}
    InChIs = getInchi(chem_file_name)
    with open(data_file_name) as h:
        for line in h:
            row = line.rstrip().split('\t')
            seq = get_seq(row[1])
            mol = None
            if row[2] in InChIs:
                mol = MolFromInchi(InChIs[row[2]])
            if (len(seq) > 0) and (mol is not None): # skip empty entries
                EC[N] = row[0]
                ID[N] = row[1]
                CP[N] = row[2]
                RX[N] = row[3]
                KM[N] = row[4]
                SEQ[N] = seq
                MOL[N] = mol
                N += 1
    return KM, MOL, SEQ

In [ ]:
# FUNCTIONS TO COMPUTE FINGERPRINTS FOR CHEM AND PROTEINS
from sklearn.preprocessing import StandardScaler, normalize

# Get Chemical Fingerprints and k-mers
def get_ChemFp(mol,binary):
    if binary:
        molFp = AllChem.GetMorganFingerprintAsBitVect(mol,2,ChemFpSize)
        Fp = [int(x) for x in list(molFp.ToBitString())]
    else:
        molFp = AllChem.GetHashedMorganFingerprint(mol,2,ChemFpSize)
        Fp = list(molFp) 
    return Fp

# k-mers for protein
def kmer(sequence,k):
    length=len(sequence)
    if k>=length:
        return # if the kmer size if greater than the lenght of the sequence
    stepsize=k-1
    i=0
    kmers=[]
    while i+stepsize<length:
        s = sequence[i:i+k]
        r = s[::-1]
        if r < s:
            s = r # only the smalest one
        kmers.append(s)
        i+=1
    return set(kmers)

def kmerset(seq,length):
    n_seq = len(seq)
    if n_seq < 1:
        return # no sequences
    kmers = kmer(seq[0],length)
    for i in range(n_seq):
        s = kmer(seq[i],length)
        kmers.update(s)
        set(kmers)
    return(kmers)   

# one hot encoding for protein
DNA = 'ATCG '
PROTEIN = 'ACDEFGHIKLMNPQRSTUVWXY '
def seq_2_onehot(seq,alphabet):
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in seq]
    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    return(onehot_encoded)

def onehot_2_seq(onehot_encoded,alphabet):
    # invert encoding
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    seq = ''
    for i in range (0,len(onehot_encoded)):
        inverted = int_to_char[np.argmax(onehot_encoded[i])]
        seq += str(inverted)
    return(seq)

# main function to compute fingerprints
def get_fingerprint(MOL,SEQ,ChemFpSize,ProtFpSize,
                    KmerSize,binary,onehot):
    SIZE = len(SEQ)
    if onehot:
        alphabet = PROTEIN
        onehot_dim=len(alphabet) 
        seqsize=0
        for i in range(SIZE):
            if len(SEQ[i]) > seqsize:
                seqsize= len(SEQ[i])
        seqsize = KmerSize*(int(seqsize/KmerSize)+1)
        ProtFpSize=int(seqsize*onehot_dim)
    else:
        onehot_dim = 0
        alphabet=kmerset(SEQ,KmerSize)
        if ProtFpSize <= 0:
            ProtFpSize = len(alphabet)
        # hash kmers in alphadic
        alphadic={x:randint(0, ProtFpSize-1) for x in alphabet} 
        
    # Get the fingerprint
    FP =  np.zeros( (SIZE, ProtFpSize) )
    FC =  np.zeros( (SIZE, ChemFpSize) )
    for i in range(SIZE):    
        FC[i] = get_ChemFp(MOL[i],binary)
        if onehot:
            s_hot = np.array(seq_2_onehot(SEQ[i],alphabet))
            hotsize = s_hot.shape[0]*s_hot.shape[1]
            s_hot = s_hot.reshape(hotsize)
            s = np.zeros( (ProtFpSize) )
            for k in range(hotsize):
                s[k] = s_hot[k]
            FP[i] = s
        else:
            for k in alphabet:
                FP[i][alphadic[k]] += SEQ[i].count(k)
        if binary == False:
            FC[i] = normalize(FC[i].reshape(1,-1))
            # FP[i] = normalize(FP[i].reshape(1,-1))
    return FC, FP, onehot_dim

In [ ]:
# DEEP LEARNING MODEL FUNCTIONS
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Conv1D, LocallyConnected1D
from keras.layers import Dropout, MaxPooling1D, Flatten, Merge, RepeatVector
from keras.layers import Merge, Lambda, Reshape, multiply, concatenate
from keras.layers.normalization import BatchNormalization
os.environ['KERAS_BACKEND'] = 'tensorflow'

def CROP(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call x = crop(2,5,10)(x) to slice the second dimension
    
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func)

def CONV(inputs, input_size, latent_size, ouput_size,
             strides, filters, dropout, hidden,
             activation, loss, optimizer): 
    out = Reshape((input_size,1)) (inputs) # needed for Conv1D
    out = LocallyConnected1D(filters=filters, 
                 kernel_size=latent_size, strides=strides,
                 activation=activation) (out)
#    out = Conv1D(filters=filters, 
#                 kernel_size=latent_size, strides=strides,
#                 activation=activation) (out)
#    if hidden:
#        out = Conv1D(filters=int(filters/10), 
#                     kernel_size=int(input_size/latent_size),
#                     activation=activation) (out)
    out = Flatten() (out)    
    out = Dropout(dropout) (out)
    outputs = Dense(ouput_size, 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
    return outputs

def RNN(inputs, input_size, latent_size, ouput_size,
               dropout, hidden, activation, loss, optimizer):   

    timesteps=int(input_size/latent_size)
    out = Reshape((timesteps,latent_size)) (inputs) # needed for LSTM   
    out = LSTM(latent_size,activation=activation, return_sequences=True)(out)
    if hidden:
        out = LSTM(latent_size,activation=activation, return_sequences=True)(out)
    out = Flatten() (out)    
    out = Dropout(dropout) (out)
    outputs = Dense(ouput_size, 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
    return outputs
 


def DENSE(inputs, input_size, latent_size, ouput_size,
               dropout, hidden, activation, loss, optimizer):   
    out = Dense(latent_size, 
                kernel_regularizer = 'l1',
                activation=activation) (inputs)
    out = Dropout(dropout) (out)
    r = 1
    while r <= hidden:
        out = Dense(int(latent_size/2**r), 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
        r += 1    
    outputs = Dense(ouput_size, 
#                    kernel_regularizer = 'l1',
                    activation=activation) (out)
    return outputs

def SEQCHEM(seq_model, che_model,
            seq_input_size, che_input_size, 
            seq_latent_size, che_latent_size, 
            mix_input_size,strides, filters, 
            dropout, hidden, activation, loss, optimizer):

    # split sequences from chemicals
    inputs = Input(shape=((seq_input_size+che_input_size,)))
    seq_inputs  = CROP(1,0,seq_input_size) (inputs)
    che_inputs = CROP(1,seq_input_size,seq_input_size+che_input_size) (inputs)
    # sequence model
    if seq_model == 'conv':
        seq = CONV(inputs=seq_inputs,
                   input_size=seq_input_size, latent_size=seq_latent_size, 
                   ouput_size=int(mix_input_size/2),
                   strides = strides, filters = filters, 
                   dropout=dropout, hidden=hidden, 
                   activation=activation, loss=loss, optimizer=optimizer)
    elif seq_model == 'dense':
        seq = DENSE(inputs=seq_inputs,
                    input_size=seq_input_size, latent_size=seq_latent_size,
                    ouput_size=int(mix_input_size/2),
                    dropout=dropout, hidden=hidden,
                    activation=activation, loss=loss, optimizer=optimizer)
    elif seq_model == 'rnn':
        seq = RNN(inputs=seq_inputs,
                    input_size=seq_input_size, latent_size=seq_latent_size,
                    ouput_size=int(mix_input_size/2),
                    dropout=dropout, hidden=hidden,
                    activation=activation, loss=loss, optimizer=optimizer)
    # chemical model
    if che_model == 'dense':
        che = DENSE(inputs=che_inputs,
                     input_size=che_input_size, latent_size=che_latent_size, 
                     ouput_size=int(mix_input_size/2),
                     dropout=dropout, hidden=hidden,
                     activation=activation, loss=loss, optimizer=optimizer)
    elif che_model == 'rnn':
        che = RNN(inputs=che_inputs,
                     input_size=che_input_size, latent_size=che_latent_size, 
                     ouput_size=int(mix_input_size/2),
                     dropout=dropout, hidden=hidden,
                     activation=activation, loss=loss, optimizer=optimizer)

    # complete mixed model
    out = concatenate([seq, che], axis=-1)
    out = Dense(mix_input_size, 
#                kernel_regularizer = 'l1',
                activation=activation) (out)
    outputs = Dense(1 ,activation='linear') (out)
    tensor = Model(inputs, outputs)
    tensor.compile(loss=loss,optimizer=optimizer,metrics=['mse'])
    print('Running Tensor model with', tensor.count_params(),'parameters' )
    return tensor

In [ ]:
# LOAD DATA (WARNING MAY TAKE A WHILE...)

datafile = '2.5.1_unique.tsv' 
chemfile = 'chem_prop.tsv' # This is the MetaNetX database
KM, MOL, SEQ = read_data_file(datafile,chemfile)
print("Loaded %d sequences, chemicals, and activity values" % len(SEQ))

In [ ]:
# COMPUTE FINGERPRINT 
# THAT'S WHERE YOU SET UP FINGERPRINT METHODS AND PARAMETERS

BINARY = False
ONEHOT = False
ChemFpSize = 1024
# the following 2 parameters are ingored if ONEHOT=True
ProtFpSize = 1024 # <= 0 for unfolded fingerprint
KMERSIZE = 3 

FC, FP, onehot_dim = get_fingerprint(MOL,SEQ,ChemFpSize,ProtFpSize,
                                     KMERSIZE,BINARY,ONEHOT)
PR =  np.zeros(len(SEQ))
for i in range(len(SEQ)):
    PR[i]=KM[i]
PR = normalize(PR.reshape(1,-1))
PR = PR.reshape(len(SEQ))

print('SEQ  FP:    ', FP.shape)
print('CHEM FP:    ', FC.shape)
print('PROP:       ', PR.shape)
print('onehot dim: ', onehot_dim)

In [ ]:
# TRAIN AND EVALUTATE MODEL USING SKLEARN GRID SEARCH

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ShuffleSplit
from keras.wrappers.scikit_learn import KerasRegressor

X = np.concatenate((FP,FC),axis=1) 
Y = PR
print('sequence:',FP.shape,'+','chemical:',FC.shape,'=',X.shape,Y.shape)
XFOLD = 5

# define the grid search parameters
n_iter_search = 4
param_grid = dict(seq_model= ['rnn'], # dense, conv (best for onehot encoding), rnn (best for kmers)
                  che_model= ['rnn'], # dense, rnn (best)
                  seq_input_size= [FP.shape[1]],
                  che_input_size = [FC.shape[1]],
                  seq_latent_size = [128,256], # for cnn: [9*onehot_dim],
                  che_latent_size = [128,256],
                  mix_input_size = [8,16,32],
                  strides = [0], # for conv: [3*onehot_dim]
                  filters = [0], # used only with conv
                  dropout = [0.33],
                  hidden = [0],
                  activation = ['relu','linear'], 
                  loss = ['mse'],
                  optimizer = ['adam'],
                  epochs = [100],
                  batch_size = [100])

# create and train model
model = KerasRegressor(build_fn=SEQCHEM, verbose=True)
grid  = RandomizedSearchCV(estimator=model, param_distributions=param_grid,n_jobs=1, 
                           cv=ShuffleSplit(n_splits=5, test_size=1/XFOLD, random_state=0), 
                           scoring='r2', 
                           n_iter=n_iter_search)
grid_result = grid.fit(X, Y)

# summarize results
q2m = grid_result.cv_results_['mean_test_score']
q2d = grid_result.cv_results_['std_test_score']
r2m = grid_result.cv_results_['mean_train_score']
r2d = grid_result.cv_results_['std_train_score']
params = grid_result.cv_results_['params']
for i in range(len(params)):
    print("q2=%.2f(%.2f) r2=%.2f(%.2f) with: %r" % (q2m[i], q2d[i], r2m[i], r2d[i], params[i]))
print("Best Q2: %.2f (R2: %.2f) using %s" % (np.amax(q2m), r2m[np.argmax(q2m)],params[np.argmax(q2m)]))
print("Best R2: %.2f (Q2: %.2f) using %s" % (np.amax(r2m), q2m[np.argmax(r2m)],params[np.argmax(r2m)]))


In [ ]:
# GET THE BEST MODEL AND DO SOMETHING WITH IT
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, train_test_split, KFold, RepeatedKFold
from sklearn import datasets, linear_model 

def get_param(params):
    for k in grid_result.best_params_.keys():
        if k == 'optimizer':
            op = params[k]
        elif k == 'epochs':
            ep = grid_result.best_params_[k]
        elif k == 'hidden':
            hi = grid_result.best_params_[k]
        elif k == 'batch_size':
            bs = grid_result.best_params_[k]
        elif k == 'activation':
            ac = grid_result.best_params_[k]
        elif k == 'loss':
            lo = grid_result.best_params_[k]
        elif k == 'dropout':
            dr = grid_result.best_params_[k]
        elif k == 'filters':
            fl = grid_result.best_params_[k]
        elif k == 'strides':
            st = grid_result.best_params_[k]
        elif k == 'mix_input_size':
            mx = grid_result.best_params_[k]
        elif k == 'che_latent_size':
            cls = grid_result.best_params_[k]
        elif k == 'seq_latent_size':
            sls = grid_result.best_params_[k]
        elif k == 'che_input_size':
            cis = grid_result.best_params_[k]
        elif k == 'seq_input_size':
            sis = grid_result.best_params_[k]
        elif k == 'che_model':
            cm = grid_result.best_params_[k]
        elif k == 'seq_model':
            sm = grid_result.best_params_[k]
    return op, ep, hi, bs, ac, lo, dr, fl, st, mx, cls, sls, cis, sis, cm, sm

optimizer, epochs, hidden, batch_size, activation, loss, dropout,\
filters, strides, mix_input_size, \
che_latent_size, seq_latent_size, che_input_size,seq_input_size,\
che_model,seq_model =get_param(grid_result.best_params_)

XFOLD=10
Q2 = np.zeros(XFOLD)
R2 = np.zeros(XFOLD)
for i in range(XFOLD):
    model = SEQCHEM(seq_model, che_model,
            seq_input_size, che_input_size, 
            seq_latent_size, che_latent_size, 
            mix_input_size,strides, filters, 
            dropout, hidden, 'linear', loss, optimizer)
    X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=1/XFOLD, random_state=2*i)
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=False)
    P = model.predict(X_valid)  
    Q2[i]= r2_score(Y_valid,P)
    P = model.predict(X_train)  
    R2[i]= r2_score(Y_train,P) 
    print('i = %4d -- Q2: %.4f R2: %.4f' % (i,Q2[i],R2[i]))

s = 'Xfold:' + str(XFOLD)
s += ' Averaged Q2: %.6f ' % (np.mean(Q2))
s += ' (%.6f)' % (np.std(Q2))
s += ' Averaged R2: %.4f ' % (np.mean(R2))
s += ' (%.4f)' % (np.std(R2))
print(s)

In [ ]:
# PLOT (YT= Y predicted) = a * (XT= Y Measured) + b

import matplotlib.pyplot as plt

YP = model.predict(X)
r2 = r2_score(Y,YP) 
XT = Y.reshape(len(Y),1)
YT = YP.reshape(len(YP),1)

print(XT.shape,YT.shape)

regr = linear_model.LinearRegression()
regr.fit(XT, YT)
YP = regr.predict(XT)

# Plot outputs
plt.scatter(XT, YT,  color='black')
plt.plot(XT, YP, color='red', linewidth=1)
plt.xlabel('measured')
plt.ylabel('predicted')
s = 'R2: ' + ("%.2f" % r2)
plt.title(s)
plt.show()

In [ ]:
"""
My previous grid search results  
No regularization (BINARY)
Best R2: 0.79 (Q2: 0.47) using {'seq_input_size': 69897, 'optimizer': 'adam', 'filters': 3, 'dropout': 0.5, 'seq_latent_size': 207, 'hidden': 0, 'che_input_size': 1024, 'epochs': 200, 'strides': 69, 'mix_input_size': 8, 'activation': 'relu', 'loss': 'mse', 'che_latent_size': 64, 'batch_size': 100}

Not that stable in Q2...
Best R2: 0.79 (Q2: 0.67) using {'seq_latent_size': 207, 'seq_input_size': 69897, 'strides': 69, 'activation': 'relu', 'mix_input_size': 8, 'hidden': 0, 'optimizer': 'adam', 'epochs': 200, 'batch_size': 100, 'loss': 'mse', 'dropout': 0.5, 'filters': 3, 'che_latent_size': 64, 'che_input_size': 1024}

Restart
Best R2: 0.70 (Q2: 0.40) using {'optimizer': 'adam', 'mix_input_size': 16, 'che_latent_size': 128, 'seq_latent_size': 207, 'dropout': 0.5, 'batch_size': 100, 'filters': 16, 'hidden': 0, 'epochs': 100, 'seq_model': 'conv', 'seq_input_size': 69897, 'che_input_size': 1024, 'strides': 69, 'loss': 'mse', 'activation': 'relu', 'che_model': 'dense'}

with regularization (dense)
Best R2: 0.69 (Q2: 0.62) using {'seq_latent_size': 207, 'seq_input_size': 69897, 'strides': 69, 'activation': 'relu', 'mix_input_size': 8, 'hidden': 0, 'optimizer': 'adam', 'epochs': 200, 'batch_size': 100, 'loss': 'mse', 'dropout': 0.5, 'filters': 3, 'che_latent_size': 64, 'che_input_size': 1024}

My results  (COUNT) 
Best R2: 0.68 (Q2: 0.22) using {'seq_latent_size': 207, 'seq_input_size': 69897, 'strides': 69, 'activation': 'relu', 'mix_input_size': 8, 'hidden': 0, 'optimizer': 'adam', 'epochs': 200, 'batch_size': 100, 'loss': 'mse', 'dropout': 0.5, 'filters': 3, 'che_latent_size': 64, 'che_input_size': 1024}

My results KMERS sequence: (313, 6609) + chemical: (313, 1024) = (313, 7633) (313,)
Best R2: 0.65 (Q2: 0.38 (0.55)) using {'batch_size': 100, 'loss': 'mse', 'filters': 3, 'hidden': 0, 'che_latent_size': 64, 'optimizer': 'adam', 'dropout': 0.5, 'seq_latent_size': 1652, 'che_input_size': 1024, 'seq_input_size': 6609, 'strides': 0, 'activation': 'relu', 'mix_input_size': 8, 'epochs': 200}

My results LSTM
Best R2: 0.78 (Q2: 0.44 (0.36)) using {'optimizer': 'adam', 'mix_input_size': 8, 'che_latent_size': 64, 'seq_latent_size': 207, 'dropout': 0.5, 'batch_size': 100, 'filters': 3, 'hidden': 0, 'epochs': 100, 'seq_model': 'conv', 'seq_input_size': 69897, 'che_input_size': 1024, 'strides': 69, 'loss': 'mse', 'activation': 'relu', 'che_model': 'rnn'}

Mixed
Best R2: 0.81 (Q2: 0.42) using {'optimizer': 'adam', 'mix_input_size': 8, 'che_latent_size': 128, 'seq_latent_size': 207, 'dropout': 0, 'batch_size': 100, 'filters': 3, 'hidden': 0, 'epochs': 200, 'seq_model': 'conv', 'seq_input_size': 69897, 'che_input_size': 1024, 'strides': 69, 'loss': 'mse', 'activation': 'relu', 'che_model': 'rnn'}
Best R2: 0.83 (Q2: 0.18) using {'optimizer': 'adam', 'mix_input_size': 8, 'che_latent_size': 256, 'seq_latent_size': 207, 'dropout': 0.1, 'batch_size': 100, 'filters': 8, 'hidden': 0, 'epochs': 200, 'seq_model': 'conv', 'seq_input_size': 69897, 'che_input_size': 1024, 'strides': 138, 'loss': 'mse', 'activation': 'relu', 'che_model': 'rnn'}
Best R2: 0.80 (Q2: 0.58) using {'batch_size': 100, 'seq_model': 'conv', 'che_input_size': 1024, 'mix_input_size': 16, 'hidden': 0, 'seq_input_size': 69897, 'seq_latent_size': 207, 'che_model': 'rnn', 'filters': 12, 'optimizer': 'adam', 'che_latent_size': 256, 'dropout': 0.5, 'strides': 69, 'epochs': 100, 'activation': 'relu', 'loss': 'mse'}

Running Tensor model with 3101481 parameters
q2=-0.13(0.00) r2=0.07(0.00) with: {'che_latent_size': 256, 'che_input_size': 1024, 'optimizer': 'adam', 'mix_input_size': 8, 'filters': 12, 'epochs': 200, 'dropout': 0.5, 'loss': 'mse', 'batch_size': 100, 'hidden': 0, 'seq_model': 'conv', 'che_model': 'rnn', 'seq_latent_size': 207, 'seq_input_size': 69897, 'strides': 69, 'activation': 'linear'}
Best Q2: -0.13 (R2: 0.07) using {'che_latent_size': 256, 'che_input_size': 1024, 'optimizer': 'adam', 'mix_input_size': 8, 'filters': 12, 'epochs': 200, 'dropout': 0.5, 'loss': 'mse', 'batch_size': 100, 'hidden': 0, 'seq_model': 'conv', 'che_model': 'rnn', 'seq_latent_size': 207, 'seq_input_size': 69897, 'strides': 69, 'activation': 'linear'}
Best R2: 0.07 (Q2: -0.13) using {'che_latent_size': 256, 'che_input_size': 1024, 'optimizer': 'adam', 'mix_input_size': 8, 'filters': 12, 'epochs': 200, 'dropout': 0.5, 'loss': 'mse', 'batch_size': 100, 'hidden': 0, 'seq_model': 'conv', 'che_model': 'rnn', 'seq_latent_size': 207, 'seq_input_size': 69897, 'strides': 69, 'activation': 'linear'}
Best Q2: 0.43 (R2: 0.77) using {'seq_input_size': 69897, 'che_model': 'rnn', 'seq_latent_size': 207, 'dropout': 0.25, 'loss': 'mse', 'filters': 12, 'optimizer': 'adam', 'mix_input_size': 8, 'hidden': 0, 'strides': 69, 'epochs': 200, 'seq_model': 'conv', 'activation': 'relu', 'che_input_size': 1024, 'batch_size': 100, 'che_latent_size': 256}
Best Q2: 0.41 (R2: 0.77) using {'seq_input_size': 1024, 'che_model': 'rnn', 'seq_latent_size': 256, 'dropout': 0.33, 'loss': 'mse', 'filters': 8, 'optimizer': 'adam', 'mix_input_size': 8, 'hidden': 0, 'strides': 0, 'epochs': 100, 'seq_model': 'rnn', 'activation': 'relu', 'che_input_size': 1024, 'batch_size': 100, 'che_latent_size': 128}
Best R2: 0.79 (Q2: -0.71) using {'optimizer': 'adam', 'activation': 'relu', 'loss': 'mse', 'che_latent_size': 256, 'che_model': 'rnn', 'seq_model': 'rnn', 'che_input_size': 1024, 'epochs': 100, 'seq_latent_size': 256, 'strides': 0, 'filters': 0, 'seq_input_size': 1024, 'dropout': 0.33, 'hidden': 0, 'mix_input_size': 16, 'batch_size': 100}
Best R2: 0.95 (Q2: 0.21) using {'batch_size': 100, 'hidden': 0, 'che_input_size': 1024, 'seq_latent_size': 128, 'dropout': 0.33, 'che_latent_size': 256, 'loss': 'mse', 'epochs': 100, 'strides': 0, 'mix_input_size': 32, 'optimizer': 'adam', 'filters': 0, 'activation': 'relu', 'che_model': 'rnn', 'seq_input_size': 1024, 'seq_model': 'rnn'}
"""